In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
#from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


Matplotlib created a temporary cache directory at /tmp/matplotlib-azq0gb22 because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
snowflake.get_connection(connection_name="HR Attrition conn")
df = get_dataframe("Final_HR_Attrition_Data")
df

Existing connection object fetched: <snowflake.connector.connection.SnowflakeConnection object at 0x7f84183d8040>
Please close the connection after use!
Reading dataframe from snowflake native connector


,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,68271.546,3,8,116,1984,40,occupational,F,Black,Childrens,...,15,False,Travel_Rarely,High,Low,Married,11,Excellent,Medium,Best
1,70471.235,3,37,18,1992,32,nurse,F,Hispanic,Psychiatric,...,10,True,Travel_Rarely,High,Medium,Married,11,Outstanding,Low,Good
2,53490.238,2,47,16,1988,36,emt,F,White,Psychiatric,...,14,False,Travel_Rarely,Very High,Low,Single,19,Good,Low,Bad
3,47749.212,1,40,32,1983,41,nurse,M,Hispanic,Acute Care Hospitals,...,3,False,Travel_Frequently,High,High,Single,13,Low,Medium,Bad
4,61379.954,1,29,11,1992,32,occupational,F,Hispanic,Psychiatric,...,1,True,Travel_Frequently,High,Medium,Married,19,Low,Medium,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,69870.670,2,42,14,1995,29,nurse,M,Black,Acute Care - Department of Defense,...,6,False,Travel_Rarely,Medium,Very High,Divorced,13,Excellent,Low,Good
299996,79047.028,5,106,164,1975,49,technologist,M,Hispanic,Acute Care - Department of Defense,...,6,True,Travel_Rarely,Low,Medium,Divorced,14,Excellent,Medium,Best
299997,51505.375,1,12,69,1985,39,social,F,Multiple,Acute Care - Department of Defense,...,9,False,Travel_Rarely,Low,Medium,Married,18,Good,Very High,Best
299998,64988.218,2,9,76,1985,39,occupational,F,API,Childrens,...,14,False,Non-Travel,Very High,Very High,Single,15,Outstanding,Low,Better


In [6]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [7]:
df = df.dropna()

In [8]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SCHOOL_END_DATE              0
JOB_START_DATE               0
JOB_END_DATE                 0
USER_ID                      0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [9]:
df = df.drop(["USER ID", "JOB START DATE", "JOB END DATE", "SCHOOL END DATE"], axis = 1)

KeyError: "['USER ID', 'JOB START DATE', 'JOB END DATE', 'SCHOOL END DATE'] not found in axis"

In [ ]:
CATEGORICAL_COLUMNS = ["MAPPED ROLE CLEAN","SEX", "ETHNICITY","HOSPITAL TYPE", "HOSPITAL OWNERSHIP","COMPANY NAME","CITY","STATE","DISTANCE", 
                       "DEGREE CLEAN","BUSINESS TRAVEL","ENVIRONMENT SATISFACTION","JOB SATISFACTION","MARTIAL STATUS","PERFORMANCE RATING","RELATIONSHIP SATISFACTION","WORK LIFE BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE MONTHS", "MONTHS AFTER COLLEGE", "BIRTH YEAR","AGE","USER ID", "OVER TIME HOURS", "PERCENTAGE SALARY HIKE",]
Date_COLUMNS = ['SCHOOL END DATE', 'JOB START DATE', 'JOB END DATE']
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]